# 12 Steps to Naiver-Stokes

The final two steps in this interactive module teaching beginning CFD with Python will both solve the Naiver-Stokes equations in two dimensions, but with different boundary conditions.

The momentum equation in vector form for a velocity field $\vec{v}$ is :
$$
\dfrac{\partial{\vec{v}}}{\partial{t}}+\big(\vec{v}\cdot\nabla\big)\vec{v} = -\dfrac{1}{\rho}\nabla{p}+\nu\nabla^2\vec{v}
$$

This represents three scalar equations, one for each velocity component$(u, v, w)$. But we will solve it in two dimensions, so where will be two scalar equations.

Remember the continuity equation? This is where in Poisson equation for pressure comes in!

## Step11: Cavity Flow with Navier-Stokes

Here is the system of differential equations: two equations for the velocity components $(u, v)$ and one equation for pressure:

$$
\dfrac{\partial{u}}{\partial{t}}+u\big(\dfrac{\partial{u}}{\partial{x}}+\dfrac{\partial{u}}{\partial{y}}\big)=-\dfrac{1}{\rho}\dfrac{\partial{p}}{\partial{x}}+\nu\big(\dfrac{\partial^2{u}}{\partial{x^2}}+\dfrac{\partial^2{u}}{\partial{y}^2}\big)
$$
